In [2]:
import os 
os.getcwd()

'c:\\kiosks_\\main\\components\\speech_recognition'

In [3]:
os.chdir('C:/kiosks_')

In [16]:
from main.functions.common_function import read_yaml_file, create_directories
from pathlib import Path
class speech_recog_config:
    def __init__(self):
        self.config = read_yaml_file(Path('config/config.yaml'))
        self.params = read_yaml_file(Path('config/params.yaml'))

    def speech_recog_module(self):
        config = self.config.local_models
        create_directories([config.speech_recog_model])

        sppech_recog_config = {
            'model_path': Path(config['speech_recog_model']),
            'model': config['speech_to_text_model'],
            'duration': self.params['duration'],
            'sample_rate': self.params['sampling_rate'],
            'channel': self.params['channel'],
        }
        
        return sppech_recog_config
        


In [26]:

from faster_whisper import WhisperModel
import torch
import sounddevice
from scipy.io.wavfile import write


class speech_recog_compo:
    def __init__(self, config, parmas):
        # obj = speech_recog()
        # self.configs = obj.speech_recog_module()
        # self.model_path = self.configs['model_path']
        #  print(self.configs)
        self.config = config
        self.params = parmas


    def download_model(self):
        os.environ["HF_HUB_DISABLE_SYMLINKS"] = "1"  # <-- ADD THIS LINE
        if torch.cuda.is_available():
            device = "cuda"
        else:
            device = "cpu"
        
        model_size = self.config['model']
        model = WhisperModel(model_size, download_root= self.config['model_path'], device=device, compute_type="float32")
        print('model succefully downloaded', model)

    
    def record_and_transcribe_audio(self):
        print('Recording audio...')
        audio = sounddevice.rec(int(self.params['duration'] * self.params['sample_rate']), samplerate=self.params['sample_rate'], channels=1, dtype='int16')
        sounddevice.wait()
        write('recording.wav', self.params['sample_rate'], audio)
        print("✅ Recording saved to", "recording.wav")
        model = WhisperModel(self.config['model'], compute_type="float32")  # use float16 if GPU available

        segments, info = model.transcribe("recording.wav", beam_size=5)
        print("🌐 Detected Language:", info.language)
        print("📝 Transcription:")
        for segment in segments:
            print(f"[{segment.start:.2f}s -> {segment.end:.2f}s] {segment.text}")
    

In [27]:
obj = speech_recog_config()
aa = obj.speech_recog_module()

created directory at: models/speech_recog_model


In [28]:

f = speech_recog_compo(aa, aa)
f.download_model()
f.record_and_transcribe_audio()


model succefully downloaded <faster_whisper.transcribe.WhisperModel object at 0x0000026427429310>
Recording audio...
✅ Recording saved to recording.wav


c:\kiosks_\myenv\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--Systran--faster-whisper-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


🌐 Detected Language: en
📝 Transcription:


In [29]:
import sounddevice as sd
from scipy.io.wavfile import write

def record_audio(filename="recording.wav", duration=5, samplerate=16000):
    print("🎤 Recording...")
    audio = sd.rec(int(duration * samplerate), samplerate=samplerate, channels=1, dtype='int16')
    sd.wait()
    write(filename, samplerate, audio)
    print("✅ Recording saved to", filename)


In [ ]:
from faster_whisper import WhisperModel

def transcribe_audio(filename="recording.wav"):
    model = WhisperModel("base", compute_type="float32")  # use float16 if GPU available
    # model = WhisperModel("large-v3", compute_type="float32")  # use float16 if GPU available

    segments, info = model.transcribe(filename, beam_size=5)
    print("🌐 Detected Language:", info.language)
    print("📝 Transcription:")
    for segment in segments:
        print(f"[{segment.start:.2f}s -> {segment.end:.2f}s] {segment.text}")




if __name__ == "__main__":
    record_audio(duration=8)  # speak something in any language
    transcribe_audio("recording.wav")



🎤 Recording...
✅ Recording saved to recording.wav
🌐 Detected Language: en
📝 Transcription:
[0.00s -> 4.00s]  Hello, hello, hello


In [24]:
import openai
from dotenv import load_dotenv
import os
import sounddevice
from scipy.io.wavfile import write
load_dotenv()
client = openai.OpenAI(api_key=os.getenv('OPENAI_API_KEY'))  # new client instance


In [ ]:
def record_audio(filename="recording_api_whisper.wav", duration=5, samplerate=16000):
    print("🎤 Recording...")
    audio = sounddevice.rec(int(duration * samplerate), samplerate=samplerate, channels=1, dtype='int16')
    sounddevice.wait()
    write(filename, samplerate, audio)
    print("✅ Recording saved to", filename)


def transcribe_with_openai(filename="recording_api_whisper.wav"):
    with open(filename, "rb") as audio_file:
        transcript = client.audio.transcriptions.create(
            model="whisper-1",
            file=audio_file,
            response_format="text"
        )
    print("📝 Transcription:", transcript)


In [26]:
if __name__ == "__main__":
    record_audio(duration=5)  # from earlier
    transcribe_with_openai("recording_api_whisper.wav")


🎤 Recording...
✅ Recording saved to recording_api_whisper.wav
📝 Transcription: 



<!-- Windows PowerShell
Copyright (C) Microsoft Corporation. All rights reserved.

Install the latest PowerShell for new features and improvements! https://aka.ms/PSWindows

PS C:\kiosks_\config> cd..
PS C:\kiosks_> myenv/Scripts/activate
(myenv) PS C:\kiosks_> & C:/kiosks_/myenv/Scripts/python.exe c:/kiosks_/main/components/speech_recognition/speech_recog.py
Traceback (most recent call last):
  File "c:\kiosks_\main\components\speech_recognition\speech_recog.py", line 1, in <module>
    from main.components.speech_recognition.config.speech_recog_config import speech_recog_config
ModuleNotFoundError: No module named 'main'
(myenv) PS C:\kiosks_> $PYTHONPATH = 'C:/kiosks_'
(myenv) PS C:\kiosks_> dir


    Directory: C:\kiosks_


Mode                 LastWriteTime         Length Name
----                 -------------         ------ ----
d-----        05-04-2025     17:13                artifacts
d-----        12-04-2025     21:47                config
d-----        11-04-2025     17:50                main
d-----        12-04-2025     19:30                models
d-----        07-04-2025     15:07                myenv
-a----        05-04-2025     12:13           3617 .gitignore
-a----        11-04-2025     15:35           9659 app.py
-a----        05-04-2025     12:13          11558 LICENSE
-a----        05-04-2025     12:13              9 README.md
-a----        12-04-2025     23:52         256044 recording.wav
-a----        12-04-2025     21:54            171 requirements.txt


(myenv) PS C:\kiosks_> cd main
(myenv) PS C:\kiosks_\main> dir


    Directory: C:\kiosks_\main


Mode                 LastWriteTime         Length Name
----                 -------------         ------ ----
d-----        11-04-2025     16:26                components
d-----        09-04-2025     14:36                functions
d-----        11-04-2025     17:50                models
d-----        11-04-2025     15:35                pipelines
d-----        05-04-2025     19:47                predifined


(myenv) PS C:\kiosks_\main> cd componets
cd : Cannot find path 'C:\kiosks_\main\componets' because it does not exist.
At line:1 char:1
+ cd componets
+ ~~~~~~~~~~~~
    + CategoryInfo          : ObjectNotFound: (C:\kiosks_\main\componets:String) [Set-Location], ItemNotFoundException
    + FullyQualifiedErrorId : PathNotFound,Microsoft.PowerShell.Commands.SetLocationCommand

(myenv) PS C:\kiosks_\main> cd components
(myenv) PS C:\kiosks_\main\components> dir


    Directory: C:\kiosks_\main\components


Mode                 LastWriteTime         Length Name
----                 -------------         ------ ----
d-----        11-04-2025     14:58                menu_maneger
d-----        12-04-2025     23:54                speech_recognition


(myenv) PS C:\kiosks_\main\components> cd speech_recognition
(myenv) PS C:\kiosks_\main\components\speech_recognition> dir


    Directory: C:\kiosks_\main\components\speech_recognition


Mode                 LastWriteTime         Length Name
----                 -------------         ------ ----
d-----        12-04-2025     23:53                config
-a----        12-04-2025     23:18         256044 recording.wav
-a----        12-04-2025     23:56           1941 speech_recog.py
-a----        12-04-2025     23:52           8566 trial.ipynb


(myenv) PS C:\kiosks_\main\components\speech_recognition> python speech_recog.py
Traceback (most recent call last):
  File "C:\kiosks_\main\components\speech_recognition\speech_recog.py", line 1, in <module>
    from main.components.speech_recognition.config.speech_recog_config import speech_recog_config
ModuleNotFoundError: No module named 'main'
(myenv) PS C:\kiosks_\main\components\speech_recognition> $env:PYTHONPATH = 'C:/kiosks_'
(myenv) PS C:\kiosks_\main\components\speech_recognition> python speech_recog.py
Traceback (most recent call last):
  File "C:\kiosks_\main\components\speech_recognition\speech_recog.py", line 45, in <module>
    obj = speech_recog_config()
          ^^^^^^^^^^^^^^^^^^^^^
  File "C:\kiosks_\main\components\speech_recognition\config\speech_recog_config.py", line 5, in __init__
    self.config = read_yaml_file(Path('config/config.yaml'))
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\kiosks_\main\functions\common_function.py", line 15, in read_yaml_file
    with open(path) as file:
         ^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: 'config\\config.yaml'
(myenv) PS C:\kiosks_\main\components\speech_recognition> python speech_recog.py
Traceback (most recent call last):
  File "C:\kiosks_\main\components\speech_recognition\speech_recog.py", line 45, in <module>
    obj = speech_recog_config()
          ^^^^^^^^^^^^^^^^^^^^^
  File "C:\kiosks_\main\components\speech_recognition\config\speech_recog_config.py", line 5, in __init__
    self.config = read_yaml_file(Path('config/config.yaml'))
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\kiosks_\main\functions\common_function.py", line 15, in read_yaml_file
    with open(path) as file:
         ^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: 'config\\config.yaml'
(myenv) PS C:\kiosks_\main\components\speech_recognition> cd..
(myenv) PS C:\kiosks_\main\components> cd..
(myenv) PS C:\kiosks_\main> cd..
(myenv) PS C:\kiosks_> python speech_recog.py
C:\Users\Admin\AppData\Local\Programs\Python\Python312\python.exe: can't open file 'C:\\kiosks_\\speech_recog.py': [Errno 2] No such file or directory
(myenv) PS C:\kiosks_> & C:/kiosks_/myenv/Scripts/python.exe c:/kiosks_/main/components/speech_recognition/speech_recog.py
created directory at: models/speech_recog_model
model succefully downloaded <faster_whisper.transcribe.WhisperModel object at 0x0000017D1760B830>
Recording audio...
✅ Recording saved to recording.wav
🌐 Detected Language: en
📝 Transcription:
[0.00s -> 4.00s]  Hello? Hello?
[4.00s -> 6.00s]  Hello?
(myenv) PS C:\kiosks_> -->